In [1]:
# %%
import numpy as np
from scipy.linalg import svd
import sys
sys.path.append("../")
import scipy.io as sio
import os


In [3]:
import h5py
with h5py.File('pressure_data.mat', 'r') as file:
    # Load DATA_U and transpose it
    DATA = np.array(file['/DATA']).T  # Transpose the data
print("size of DATA: ", DATA.shape)

size of DATA:  (798, 295122)


In [12]:
# %%
# Load the MAT file

# Define the number of singular values/components to keep
k = 150  # for example, keep 150 components

# Perform SVD and keep only the first k of U, S, and VT
U_reduced, S_reduced, VT_reduced = svd(DATA, full_matrices=False)
U_reduced = U_reduced[:, :k]
S_reduced = S_reduced[:k]
VT_reduced = VT_reduced[:k, :]

# Reconstruct the reduced data matrix
DATA_reduced = np.dot(U_reduced, np.diag(S_reduced))

# Now you can slice the DATA as before
X = DATA_reduced[:-1]  # All rows except the last
Y = DATA_reduced[1:]   # All rows except the first

print("size of DATA reduced: ", DATA_reduced.shape)
print("size of X: ", X.shape)
# Clean up
# del temp

len_all = X.shape[0]
data_x_train = X[:int(0.7*len_all),:]
data_x_valid = X[int(0.7*len_all)+1:,:]

data_y_train = Y[:int(0.7*len_all),:]
data_y_valid = Y[int(0.7*len_all)+1:,:]

data_train = [data_x_train, data_y_train]
data_valid = [data_x_valid, data_y_valid]

size of DATA reduced:  (798, 150)
size of X:  (797, 150)


In [15]:
# %%
from solver_resdmd_tf import KoopmanNN, KoopmanSolver

# basis_function = PsiNN(layer_sizes=[100,100,100], n_psi_train=n_psi_train)
# N_dict = n_psi_train + 3
# # Dynamically import the solver module based on solver_index
# filename = f'solver_{solver_index}_outputs_{N_dict}basis.mat'
# solver_module = __import__(f"solver_{solver_index}", fromlist=['KoopmanDLSolver'])
# KoopmanDLSolver = getattr(solver_module, 'KoopmanDLSolver')
n_psi=99
basis_function = KoopmanNN(layer_sizes=[120,120,120], n_psi_train=n_psi)

# Using the dynamically imported solver
solver = KoopmanSolver(dic=basis_function, 
                            target_dim=np.shape(data_train)[-1], 
                            reg=0.1)
solver.build(data_train=data_train, 
                data_valid=data_valid, 
                epochs=100, 
                batch_size=500, 
                lr=1e-4, 
                log_interval=10, 
                lr_decay_factor=.8)

# Results from solver
evalues = solver.eigenvalues
efuns_X = solver.eigenfunctions(X)
efuns_Y = solver.eigenfunctions(Y)
N_dict = np.shape(evalues)[0]    
Psi_X = solver.get_Psi_X().numpy()
Psi_Y = solver.get_Psi_Y().numpy()
# Koopman_matrix_K = solver.K.numpy()
# kpm_modes = solver.compute_mode()
kpm_modes = np.dot(VT_reduced.T, solver.compute_mode())

# Prepare data to save
outputs = {
    'evalues': evalues,
    'efuns_X': efuns_X,
    'efuns_Y': efuns_Y,
    'N_dict': N_dict,
    'Psi_X': Psi_X,
    'Psi_Y': Psi_Y,
    # 'K': Koopman_matrix_K,
    'kpm_modes': kpm_modes,
}

Outer Epoch 1/100
Epoch 1/2
2/2 [==============================] - 1s 293ms/step - loss: 205.6075 - val_loss: 58590.3094
Epoch 2/2
2/2 [==============================] - 0s 31ms/step - loss: 205.6075 - val_loss: 58590.2966
Outer Epoch 2/100
Epoch 1/2
2/2 [==============================] - 0s 53ms/step - loss: 205.6075 - val_loss: 58590.3173
Epoch 2/2
2/2 [==============================] - 0s 28ms/step - loss: 205.6075 - val_loss: 58590.3415
Outer Epoch 3/100
Epoch 1/2
2/2 [==============================] - 0s 57ms/step - loss: 205.6075 - val_loss: 58590.3427
Epoch 2/2
2/2 [==============================] - 0s 30ms/step - loss: 205.6075 - val_loss: 58590.3204
Outer Epoch 4/100
Epoch 1/2
2/2 [==============================] - 0s 58ms/step - loss: 205.6075 - val_loss: 58590.3046
Epoch 2/2
2/2 [==============================] - 0s 32ms/step - loss: 205.6075 - val_loss: 58590.2999
Outer Epoch 5/100
Epoch 1/2
2/2 [==============================] - 0s 60ms/step - loss: 205.6075 - val_loss: 58

In [16]:
print(efuns_X.shape)
print(efuns_Y.shape)

(797, 250)
(797, 250)


In [17]:
folder_path = 'data/turbulence_data'  # Adjust the path as needed if you want a different location
os.makedirs(folder_path, exist_ok=True)
# Save the outputs to a .mat file named according to the number of basis functions, inside the "data" folder
file_path = os.path.join(folder_path, f'turbulence_resdmd_{N_dict}basis.mat')
sio.savemat(file_path, outputs)
print(f'Saved outputs for n_psi_train={n_psi} to {file_path}')

Saved outputs for n_psi_train=99 to data/turbulence_data\turbulence_resdmd_250basis.mat
